In [5]:
import numpy
from transformers import pipeline
import pandas as pd
import os
from datetime import datetime 

In [15]:
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
# model_id = "cardiffnlp/twitter-roberta-base-sentiment-latest"

sentiment_pipeline = pipeline("sentiment-analysis", model=model_id)

In [7]:
path = r'C:\Users\aefig\OneDrive\Escritorio\Tec\07_6to Trimestre\02_ProyectoIntegrador\NewsRegister'

files = []

for directory in os.listdir(path):
    if directory[-4:] != '.csv' and directory in ['2021']: #ONLY 1 YEARS
        for file in os.listdir(path + '\\' + directory):
            files.append(path + '\\' + directory + '\\' + file)

In [24]:
# Function to obtain sentiment analysis
def sentimentNews_text(row):
    text1 = row[f'Text_1'] if row[f'Text_1'][:5] != 'ERROR' else ''
    text2 = row[f'Text_2'] if row[f'Text_2'][:5] != 'ERROR' else ''
    text3 = row[f'Text_3'] if row[f'Text_3'][:5] != 'ERROR' else ''
    text4 = row[f'Text_4'] if row[f'Text_4'][:5] != 'ERROR' else ''
    text5 = row[f'Text_5'] if row[f'Text_5'][:5] != 'ERROR' else ''
    
    text = []
    
    if text1 != '':
        text.append(text1)
    if text2 != '':
        text.append(text2)
    if text3 != '':
        text.append(text3)
    if text4 != '':
        text.append(text4)
    if text5 != '':
        text.append(text5)
    
    AddedSentiment = 0
    
    try:
        results = sentiment_pipeline(text)

        for result in results:
            if result['label'] == 'POSITIVE':
                AddedSentiment += result['score']
            elif result['label'] == 'NEGATIVE':
                AddedSentiment -= result['score']
    except:
        AddedSentiment = 999
            
    return AddedSentiment

In [25]:
day = 0

for file in files[12:13]:
#     start_time = datetime.now()

    day += 1
    if day % 36 == 0:
            print(f"{int(day/3.6)}%")
            
    print(f"\t{file}")       

    df = pd.read_csv(file)
    df = df.fillna('')
    for i in range(1, 6):
        df[f"Text_{i}"] = df[f"Text_{i}"].str[:512]
        df[f"Name_{i}"] = df[f"Name_{i}"].str[:512]

    
    df['AddedSentiment_text'] = df.apply(lambda x: sentimentNews_text(x), axis=1)
    
    df_count = df[['Symbol', 'AddedSentiment_text']]
    df_count = df_count.set_index('Symbol')
    df_count = df_count.T
    df_count.insert(loc=0, column='Date', value=file[-14:][:10])
    
    path = r'C:\Users\aefig\OneDrive\Escritorio\Tec\07_6to Trimestre\02_ProyectoIntegrador'
    
    if 'NewsRegisterSentiment.csv' in os.listdir(path):
        header_flag = False
    else:
        header_flag = True

    df_count.to_csv(r"C:\Users\aefig\OneDrive\Escritorio\Tec\07_6to Trimestre\02_ProyectoIntegrador\NewsRegisterSentiment.csv", mode='a', index=False, header=header_flag)
    
#     end_time = datetime.now() 
#     print(f"Done in {(end_time - start_time).total_seconds()}")

	C:\Users\aefig\OneDrive\Escritorio\Tec\07_6to Trimestre\02_ProyectoIntegrador\NewsRegister\2021\2021-01-13.csv
